# Modularbeit Künstliche Intelligenz
## Name: Arrif Sondjalim
## Studiengang: DE 
## Matrikelnummer: 26959622

In [ ]:
import random
import time
from IPython.display import clear_output

In [ ]:
'''
Definition von Objekten und Feldtypen
'''
WALL = '#'
SPACE = ' '
NEST = 'N'
FOOD = 'o'
ANT = 'A'
ANT_WITH_FOOD = 'C'

In [ ]:
'''
Ameisenklasse
'''
class Ant:
    def __init__(self, environment, x, y):
        self.x = x # X-Position in der Environment
        self.y = y # Y-Position in der Environment
        self.environment = environment
        self.carry = '' # Was trägt die Ameise gerade, '' für nichts
        
    # eine Ameise kann sich ein Feld nach oben, unten, links oder rechts bewegen    
    def walk(self, dx, dy):
        if abs(dx) + abs(dy) > 1:
            return
        if self.environment.getFloor(self.x + dx, self.y + dy) != WALL:
            self.x += dx
            self.y += dy
    
    # eine Ameise kann wahrnehmen, auf welchem Feld sie sich befindet
    # - einem leeren Feld: SPACE
    # - dem Nest NEST
    # - einem Feld mit Essen FOOD
    def sense(self):
        return self.environment.getFloor(self.x, self.y)
    
    # trägt die Ameise gerade Essen mit sich?
    def carryingFood(self):
        return self.carry == FOOD
    
    # nehme Essen auf
    def take(self):
        if self.carry == '' and self.sense() == FOOD:
            self.carry = FOOD
            self.environment.setFloor(self.x, self.y, SPACE)
      
    # lasse das Essen fallen
    def drop(self):
        if self.carry != '':
            if self.sense() == NEST:
                self.environment.setFloor(self.x, self.y, SPACE)
            else:
                self.environment.setFloor(self.x, self.y, self.carry)
            self.carry = ''
    
    def random_walk(self):
        """ Beschreibung: Fuehrt eine zufällige Bewegung der Ameise durch. """
        # Zufällige Bewegungsrichtung: Oben, Rechts, Unten, Links
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)] 
        dx, dy = random.choice(directions)
        self.walk(dx, dy)

    def act(self):
        """ Beschreibung: Methode, die die Nahrungssuche einer Ameise basierend auf Zufaellen automatisiert. """
        # Prüfe, ob die Ameise auf Nahrung oder dem Nest steht:
        here = self.sense()
        if self.carryingFood():
            if here == NEST:
                self.drop()  # Essen im Nest ablegen
                return
        else:
            if here == FOOD:
                self.take()  # Essen aufnehmen
                return

        # Bewegung falls keine Aktionen bzgl. Essen/Nest vorgenommen wurden
        self.random_walk()

In [ ]:
class Environment:
    def __init__(self, conf):    # lade eine Umgebung aus einer textuellen Beschreibung
        self.dimX = len(conf[0]) # Breite der Umgebung
        self.dimY = len(conf)    # Höhe der Umgebung
        self.floor = [[conf[j][i] if conf[j][i] != ANT else SPACE for i in range(self.dimX)] for j in range(self.dimY)]
        self.ant = Ant(self, 1, 1)
        for j in range(self.dimY):
            for i in range(self.dimX):
                if conf[j][i] == ANT:
                    self.ant.x = i
                    self.ant.y = j
        
    def setFloor(self, x, y, w): # setze ein Feld in der Umgebung
        self.floor[y][x] = w
        
    def getFloor(self, x, y): # gebe das Feld der Umgebung zurück
        return self.floor[y][x]
    
    def print(self): # drucke eine textuelle Beschreibung der Umgebung aus
        for y in range(self.dimY):
            for x in range(self.dimX):
                if self.ant.x == x and self.ant.y == y:
                    if self.ant.carryingFood():
                        print(ANT_WITH_FOOD, end='')
                    else:
                        print(ANT, end='')
                else:
                    print(self.getFloor(x, y), end='')
            print()
            
    def anyNestsLeft(self): # gibt es noch ein Nest, welches noch kein Futter erhalten hat?
        for y in range(self.dimY):
            for x in range(self.dimX):
                if self.floor[y][x] == NEST:
                    return True
        return False
    
    def tick(self): # führe einen Simulationsschritt durch
        self.ant.act()
        
    def simulate(self, verbose=False): # simuliere die Umgebung solange es noch Nester ohne Futter gibt
        steps = 0
        while self.anyNestsLeft():
            self.tick()
            steps += 1
            if verbose:
                self.print()
                time.sleep(0.05)
                clear_output(wait=True)
        return steps

In [ ]:
scenario1 = [
"##########",
"#N       #",
"#        #",
"#        #",
"#        #",
"#    A   #",
"#        #",
"#        #",
"#       o#",
"##########"
]

scenario2 = [
"##########",
"#N       #",
"######   #",
"#A#      #",
"#   ######",
"#   #    #",
"#   # # ##",
"#   # #  #",
"#     # o#",
"##########"
]

env = Environment(scenario1)
env.simulate(True)
ant1 = Ant(env, 1, 1)
ant1.act()

# Hinweis: Dieses Szenario hat eine hohe Rechenzeit zur Folge. Idealfall zwischen 2 und 6 Minuten
umgebung = Environment(scenario2)
umgebung.simulate(True)
ant2 = Ant(env, 1, 1)
ant2.act()

In [ ]:
def simulate_multiple_times(environment_conf: list[str], runs=1000):
    """
    Beschreibung: Implementierung einer Funktion, die den Mittelwert an zurueckgelegten Schritten berechnet
    Parameter: `environment_conf`: Eine Szenario als Liste mit Strings als Elementen, `runs`: Anzahl an Wiederholungen mit dem Default-Wert 1000
    Returns: Mittelwert an zurueckgelegten Schritten
    """
    total_steps = 0
    for _ in range(runs):
        env = Environment(environment_conf)
        steps = env.simulate(verbose=False)
        total_steps += steps
    return total_steps / runs

# Ermittlung des arithmetischen Mittelwerts der Anzahl an Schritten 
average_steps_scenario1 = simulate_multiple_times(scenario1)
print(f"Mittlere Anzahl von Schritten für scenario1: {average_steps_scenario1}")

# Ermittlung des arithmetischen Mittelwerts der Anzahl an Schritten 
average_steps_scenario2 = simulate_multiple_times(scenario2)
print(f"Mittlere Anzahl von Schritten für scenario2: {average_steps_scenario2}")

# Antworten

## **Aufabe 3**

1. **Hindernisse**: Die beiden Szenarien unterscheiden sich stark. Beide Szenarien werden als Listen, 
mit unterschiedlicher Komplexitaet deren einzelnen Elemente dargestellt. Der offensichtlichste Unterschied ist die Existenz von Hindernissen 
im zweiten Szenario, die das Erreichen des Ziels erschweren. Im ersten Szenario gibt es keine Hindernisse, die den Weg blockieren.

2. **Rechenzeit**: Wie schon im vorherigen Aspekt bezueglich den Hindernissen erwähnt, beeinflusst die Komplexitaet der
Hindernisse die Suche maßgeblich. Im ersten Szenario ist die Pfadfindung einfach und geradeaus, wohingegen im zweiten Szenario die Ameise einen 
komplexeren Pfad durch das Labyrinth von Waenden finden muss, was in einer hoeheren Rechenzeit resultiert.
